In [26]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath + 'dataImport/')
from utils import workingDataPath, getWorkingHRDfParquet
import pandas as pd

appleHRDf = getWorkingHRDfParquet('apple')
polarHRDf = getWorkingHRDfParquet('polar')
fitbitHRDf = getWorkingHRDfParquet('fitbit')

c:\Users\abhik\Documents\GitHub\dataImport\utils.py:21: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  dfSoFar = pd.concat([dfSoFar, pd.read_parquet(workingDataHRPath + dataFileName)])
c:\Users\abhik\Documents\GitHub\dataImport\utils.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfSoFar = pd.concat([dfSoFar, pd.read_parquet(workingDataHRPath + dataFileName)])
c:\Users\abhik\Documents\GitHub\dataImport\utils.py:21: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, 

In [27]:
# intersections kinda look like this
#    |-|   |---|   |-|   |-|      |---|   |--------|
# |-|    |---|    |---|      |-|    |---|  |-| |-|
testGroups1 = [[40, 50], [70, 90], [110, 120], [140, 150], [180, 200], [220, 280]]
testGroups2 = [[20, 30], [60, 80], [100, 130], [160, 170], [190, 210], [230, 240], [250,270]]

In [67]:
# returns a list of [[startTimestamp, endTimestamp], ...] 
# for every group with samples always less than maxDelta apart
# and at least coveres minGroupTime
def getGroups(series, maxDelta = 20, minGroupTime = pd.Timedelta(minutes=5)):
    # add a column that is the time to next reading
    timesSeries = pd.Series(series.index)
    betweenMesures = ((timesSeries.shift(-1) - timesSeries)).dt.total_seconds()
    timeToNextDF = pd.DataFrame(timesSeries)
    timeToNextDF["timeToNextReading"]  =  betweenMesures
    # print(timeToNextDF.head(2))

    #filter out time to next reading that are too high
    ttnrGreaterThanThresh = timeToNextDF[timeToNextDF["timeToNextReading"] < maxDelta]
    # print(ttnrGreaterThanThresh.head(2))

    #the indexes that have been removed indicate the boundaries of contiguious sections
    index_diff = ttnrGreaterThanThresh.index.to_series().diff()
    boundaryIndicators = index_diff[index_diff > 1].index.to_list()
    # print(boundaryIndicators)

    #add each bounded area to groupBoundaries, if it lasts longer than the minGroup time
    clusterStartIndex = 0
    groupBoundaries = []
    for bii in range(len(boundaryIndicators)): #bii being boundaryIndicatorIndex
        startTime = timeToNextDF["sampleDT"].iloc[clusterStartIndex]
        endTime = timeToNextDF["sampleDT"].iloc[boundaryIndicators[bii] - 1]
        # print( endTime - startTime)
        if endTime - startTime >= minGroupTime:
            groupBoundaries.append([startTime, endTime])
        
        clusterStartIndex = boundaryIndicators[bii]

    
    return groupBoundaries

In [68]:
polarGroups = getGroups(polarHRDf, 5)
print(len(polarGroups))

30


In [58]:
print(polarHRDf.iloc[160:165])
print(polarHRDf.iloc[1300:1305])

                          value
sampleDT                       
2019-10-27 15:55:28-07:00    69
2019-10-27 15:55:29-07:00    67
2019-10-27 15:55:30-07:00    66
2019-10-27 16:16:57-07:00    95
2019-10-27 16:16:58-07:00    97
                          value
sampleDT                       
2019-10-27 16:35:54-07:00    86
2019-10-27 16:35:55-07:00    86
2019-10-28 18:26:07-07:00    85
2019-10-28 18:26:08-07:00    82
2019-10-28 18:26:09-07:00    80


In [66]:
def printHRDfGroupsInfo(HRDf, groups, index):
    groupVals = HRDf[(HRDf.index > groups[index][0]) & (HRDf.index < groups[index][1])]
    print(groups[index][1] - groups[index][0])
    print(len(groupVals))

print(polarGroups[1])
printHRDfGroupsInfo(polarHRDf, polarGroups, 1)
print(polarGroups[2])
printHRDfGroupsInfo(polarHRDf, polarGroups, 2)

print(polarGroups[3])
printHRDfGroupsInfo(polarHRDf, polarGroups, 3)

print(polarGroups[4])
printHRDfGroupsInfo(polarHRDf, polarGroups, 4)


[Timestamp('2019-10-28 18:26:07-0700', tz='pytz.FixedOffset(-420)'), Timestamp('2019-10-28 18:51:46-0700', tz='pytz.FixedOffset(-420)')]
0 days 00:25:39
1538
[Timestamp('2019-10-28 19:25:14-0700', tz='pytz.FixedOffset(-420)'), Timestamp('2019-10-28 19:31:59-0700', tz='pytz.FixedOffset(-420)')]
0 days 00:06:45
404
[Timestamp('2019-10-28 19:35:43-0700', tz='pytz.FixedOffset(-420)'), Timestamp('2019-10-28 23:59:06-0700', tz='pytz.FixedOffset(-420)')]
0 days 04:23:23
15802
[Timestamp('2019-10-29 00:16:21-0700', tz='pytz.FixedOffset(-420)'), Timestamp('2019-10-29 07:35:05-0700', tz='pytz.FixedOffset(-420)')]
0 days 07:18:44
26323


In [69]:
appleGroups = getGroups(appleHRDf, 600)
print(len(appleGroups))
polarGroups = getGroups(polarHRDf, 5)
print(len(polarGroups))
fitbitGroups = getGroups(fitbitHRDf, 20)
print(len(fitbitGroups))

4492
30
24776


In [71]:
#actually the grouping function I already wrote looks great
# what I want is the intersecting function
#it just has to take in two groups and return the intersection
# that can be called iteratively for every group

def calcIntersection(groups1, groups2):

    intersectingGroups = []

    groups1i = 0
    groups2i = 0

    # go though every group in group 1
    while groups1i < len(groups1) and groups2i < len(groups2):
        # no intersctions 
        # if the end time of group 2 is less than the start time of group 1
        if groups1[groups1i][1] < groups2[groups2i][0]:
            #no intersection, go to the next group in groups1
            groups1i += 1
            continue
        
        # if the groups1 group we're looking starts after the end of the group 2 group
        if groups1[groups1i][0] > groups2[groups2i][1]:
            #no intersection, and check the next late group
            groups2i += 1
            continue
        
        # print(f"groups1[{groups1i}]{groups1[groups1i]}")
        # print(f"groups2[{groups2i}]{groups2[groups2i]}")

        # there is an intersection and it is the latest of the start times
            # and the earliest of the end times
        intersectionStart = max(groups1[groups1i][0], groups2[groups2i][0])
        intersectionEnd = min(groups1[groups1i][1], groups2[groups2i][1])
        intersectingGroups.append([intersectionStart, intersectionEnd])

        #whichever set had the group with the earliest end time, iterate that one
        if groups1[groups1i][1] <= groups2[groups2i][1]:
            groups1i += 1
        else:
            groups2i += 1
        
    return intersectingGroups



In [73]:
len(calcIntersection(appleGroups, polarGroups))

28

In [74]:
len(calcIntersection(appleGroups, fitbitGroups))

18756